In [1]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
%run input_data_check.ipynb
%run pii_detector.ipynb

In [7]:
from guardrails import Guard

In [8]:
matcher = AdvancedQueryMatcher()

guard = Guard()
guard.use_many(AdvancedPIIValidator())

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: ./Embedding_Model/all_mini_embed_model/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/c9745ed1d9f207416be6d2e6f8de32d1f16199bf


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Guard(id='WINRWE', name='gr-WINRWE', description=None, validators=[ValidatorReference(id='AdvancedPIIValidator', on='$', on_fail='exception', args=None, kwargs={})], output_schema=ModelSchema(definitions=None, dependencies=None, anchor=None, ref=None, dynamic_ref=None, dynamic_anchor=None, vocabulary=None, comment=None, defs=None, prefix_items=None, items=None, contains=None, additional_properties=None, properties=None, pattern_properties=None, dependent_schemas=None, property_names=None, var_if=None, then=None, var_else=None, all_of=None, any_of=None, one_of=None, var_not=None, unevaluated_items=None, unevaluated_properties=None, multiple_of=None, maximum=None, exclusive_maximum=None, minimum=None, exclusive_minimum=None, max_length=None, min_length=None, pattern=None, max_items=None, min_items=None, unique_items=None, max_contains=None, min_contains=None, max_properties=None, min_properties=None, required=None, dependent_required=None, const=None, enum=None, type=ValidationType(anyof

The below function serves as the main entry point for processing a user query comprehensively. It analyzes the query for similarity against predefined risky topics and sentences, evaluates risk levels, and provides both a detailed analysis and a category-wise breakdown. Based on the results, it determines whether the query is safe, high-risk, or medium-risk. For safe queries below the low-risk threshold, it marks them as approved. For high-risk queries, it blocks the request with an explanatory message. Medium-risk queries receive a cautious response, offering general educational guidance while maintaining safety. The function returns a structured response dictionary containing the query, analysis, category breakdown, status, and response message.

In [9]:
def process_query_comprehensive(query: str) -> Dict[str, Any]:
    """
    Main function to process a query with comprehensive analysis and response.
    """
    
    # Perform full query analysis
    analysis = matcher.analyze_query(query)
    
    # Get detailed breakdown by category
    category_breakdown = matcher.get_category_breakdown(query)
    
    # Initialize response dictionary
    response = {
        'query': query,
        'analysis': analysis,
        'category_breakdown': category_breakdown,
        'response': ''
    }

    # Case 1: Query is safe and similarity is below low-risk threshold
    if analysis.is_safe and analysis.highest_similarity < matcher.risk_thresholds['low_risk']:
        response['response'] = 'SAFE'
        response['status'] = 'approved'
    
    # Case 2: Query has high-risk similarity
    else:        
        if analysis.highest_similarity >= matcher.risk_thresholds['high_risk']:
            response['response'] = (
                f"I cannot assist with this request as it relates to {analysis.primary_category.replace('_', ' ')} "
                f"content. Please ensure your queries are appropriate and safe."
            )
            response['status'] = 'blocked'
        
        # Case 3: Medium-risk query - cautious guidance
        else:
            response['response'] = (
                f"I notice your query may relate to {analysis.primary_category.replace('_', ' ')} topics. "
                f"I can provide general, educational information while ensuring safety and appropriateness."
            )
            response['status'] = 'cautious'
    
    # Return final structured response
    return response


The below function processes a single user query by first running a comprehensive matcher (process_query_comprehensive) to obtain classification, analysis, and a suggested response. It then branches on the matcher's returned status: if BLOCKED, it returns a blocked response with a small analysis summary; if CAUTIOUS, it logs/prints the matcher result and returns a cautious response summary (intentionally avoiding any further guard validation); if APPROVED, it runs a guard validation (guard.validate) and records whether the guard passed or failed (calling check_guardrail_result to amend the matcher result on failure). Any unexpected exception during processing returns an "error" response with the exception text.

In [64]:
def process_query(query: str) -> Dict[str, Any]:
    # Define function signature: accepts a query string and returns a result dictionary
    """Process a single query with matcher and Guard validation."""
    # Docstring: short description of the function purpose

    try:
        # Start a try block to catch unexpected runtime errors during processing
        matcher_result = process_query_comprehensive(query)
        # Call the comprehensive matcher to analyze the query; expect a dict with keys like 'status', 'analysis', 'response'
        

        # --- BLOCKED branch ---
        if matcher_result.get("status", "").upper() == "BLOCKED":
            # Normalize status string and check if result indicates a blocked query
            analysis = matcher_result.get("analysis")
            # Extract the analysis object from the matcher result (likely a QueryAnalysis object)

            return {
                # Return a dictionary immediately for blocked queries
                "query": query,  # Echo the original query
                "status": "blocked",  # Human-friendly status
                "analysis": {
                    # Provide a compact analysis summary for callers
                    "is_safe": analysis.is_safe,
                    "risk_assessment": analysis.risk_assessment,
                    "flagged_categories": analysis.flagged_categories
                },
                "response": matcher_result.get("response", "")
                # Include the text response from the matcher if present (empty string fallback)
            }

        # --- CAUTIOUS branch ---
        elif matcher_result.get("status", "").upper() == "CAUTIOUS":
            # Check if the matcher classified the query as cautious
            # PEVENTING GUARDRAIL CHECK IF DETECTED HERE
            analysis = matcher_result.get("analysis")
            # Extract analysis object for inclusion in the returned summar

            return {
                # Return a cautious response summary to caller
                "query": query,  # Echo original query
                "status": "cautious",  # Human-friendly status
                "analysis": {
                    # Provide compact analysis summary
                    "is_safe": analysis.is_safe,
                    "risk_assessment": analysis.risk_assessment,
                    "flagged_categories": analysis.flagged_categories
                },
                "response": matcher_result.get("response", "")
                # Include matcher's response text (or empty string)
            }

        # --- APPROVED branch ---
        elif matcher_result.get("status", "").upper() == "APPROVED":
            # If matcher approves the request, perform guardrail validation
            try:
                guard.validate(query)
                # Call guard.validate to perform policy/guardrail checks on the query; may raise an exception on failure
                matcher_result["guard_validation"] = "passed"
                # If no exception raised, mark guard validation as passed in the matcher_result dict
            except Exception as e:
                # If guard validation raised an exception, capture it here
                matcher_result["guard_validation"] = f"failed - {str(e)}"
                # Annotate matcher_result with the failure reason
                matcher_result = check_guardrail_result(matcher_result)

                # Optionally mutate/augment matcher_result via check_guardrail_result to reflect guard findings

            return matcher_result
            # Return the (possibly updated) matcher_result for approved queries after guard validation

    except Exception as e:
        # Top-level exception handler: catch any unexpected error that occurred above
        return {
            # Return a standardized error response
            "query": query,  # Echo original query
            "status": "error",  # Status indicating an internal error
            "error": str(e)  # Stringified exception message for debugging
        }


In [66]:
result = process_query_batch(["hello 9876543421"])

{'query': 'hello 9876543421', 'status': 'rejected', 'reason': 'failed - validation failed for field with errors: pii risk detected: phone number detected; bank account information detected', 'message': 'Query rejected due to guardrail validation failure.'}


In [67]:
print(result)

[{'query': 'hello 9876543421', 'status': 'not_completed', 'reason': {'status': 'rejected', 'guard_validation': '', 'risk_assessment': 'not available'}, 'message': 'Query not safe to process, execution blocked.'}]


The function check_guardrail_result ensures that results returned after processing a query correctly reflect any guardrail validation outcomes.
* If the guard validation failed, the query is immediately marked as rejected with a rejection reason.
* If the query’s status is already blocked, the blocked result (including analysis) is returned as-is.
* Otherwise, the query is considered approved, and the result is returned with a passed guard validation (default if not explicitly provided).

This acts as a post-processor to normalize the query result and enforce guardrail decisions consistently.

In [62]:
def check_guardrail_result(result: Dict[str, Any]) -> Dict[str, Any]:
    """Check the processed query result for guardrail failures."""
    
    
    # Extract the current status of the result (default "unknown" if missing).
    status = result.get("status", "unknown").lower()
    
    # Extract the guard validation result (default empty string).
    # Convert to lowercase for consistent comparison.
    guard_validation = result.get("guard_validation", "").lower()
    

    # --- Case 1: Guardrail validation failed ---
    if "failed" in guard_validation:
        # If guard validation indicates failure, override the result.
        return {
            "query": result.get("query"),  # Original query text
            "status": "rejected",  # Explicit rejection status
            "reason": guard_validation,  # Store failure reason for debugging
            "message": "Query rejected due to guardrail validation failure."
            # User-facing message explaining rejection
        }

    # --- Case 2: Query was BLOCKED earlier ---
    if status == "blocked":
        # Preserve the original blocked status, analysis, and response
        return {
            "query": result.get("query"),     # Original query
            "status": "blocked",              # Keep blocked status
            "analysis": result.get("analysis"),  # Risk/analysis info from matcher
            "response": result.get("response")   # Response provided by matcher
        }

    # --- Case 3: Otherwise, treat as APPROVED ---
    return {
        "query": result.get("query"),  # Original query
        "status": "approved",  # Mark as approved since no failures/blocking found
        "response": result.get("response", ""),  # Safe response (fallback empty string)
        "guard_validation": guard_validation or "passed"
        # If guard_validation not explicitly set, assume "passed"
    }


The function call_model_safely provides a safety wrapper for calling a language model (in this case, Ollama with llama3:8b).
* It ensures that the model is always prompted with safety instructions (system message) to avoid unsafe responses.
* It tries to generate a response from the model while catching any errors.
* If the model call fails for any reason (e.g., API issue, network problem), it logs the error and returns a polite fallback message instead of crashing.

This function guarantees both safe prompting and robust error handling.

In [12]:
def call_model_safely(query: str) -> str:
    """Call the language model with safety checks."""
    try:
        # Build conversation history for the model
        messages = [
            {
                "role": "system",   # System prompt sets rules for model behavior
                "content": (
                    "You are a helpful and safe assistant. "
                    "Answer only valid, safe, and appropriate questions. "
                    "If the query involves harmful, illegal, or inappropriate content, "
                    "politely decline and explain why you cannot help. "
                    "Be clear, concise, and professional."
                )
            },
            {
                "role": "user",    # Actual user query
                "content": query
            }
        ]
        
        # Call Ollama model with structured messages
        response = ollama.chat(
            model="llama3:8b",     # Chosen model
            messages=messages,     # Messages (system + user)
            options={"temperature": 0.0}  # Deterministic output for safety
        )
        
        # Extract model's generated text from response object
        return response['message']['content']
    
    except Exception as e:
        # Return a polite fallback message to the user
        return "I apologize, but I'm unable to process your request at this time."


The function execute_if_safe is a gatekeeper for executing the model call.
* It takes in a result dictionary (likely produced by your query matcher + guard system).
* It checks if the query is safe, approved, and passed guard validation before allowing execution.
* If all checks pass → it calls call_model_safely to get the model response.
* If checks fail → it blocks execution and returns a structured reason explaining why it was not completed.

This ensures strict enforcement of safety checks before hitting the model.

In [75]:
def execute_if_safe(result: Dict[str, Any]) -> Dict[str, Any]:
    """Execute call_model_safely only if the response is safe and guard validation passed.
    Otherwise return reason for not completing."""
    
    # Extract key fields from the result dict
    analysis = result.get("analysis")                        # Analysis object/dict with risk info
    status = result.get("status", "").lower()               # Status string: approved / blocked / cautious / error

    if 'reason' in result:
        guard_validation = result['reason']
    else:
        guard_validation = result.get("guard_validation", "blocked by safety check").lower()  # Guard check result: passed / failed
    

    # Extract risk assessment text if analysis is present
    if isinstance(analysis, dict):
        risk_assessment = analysis.get("risk_assessment", "not available").lower()
    elif hasattr(analysis, "risk_assessment"):
        risk_assessment = getattr(analysis, "risk_assessment", "not available").lower()
    else:
        risk_assessment = "not available"

    
    # Condition: only allow execution if ALL checks are safe
    if (
        analysis                               # Analysis must exist
        and getattr(analysis, "is_safe", False)  # Analysis must say it's safe
        and status == "approved"                 # Must be explicitly approved
        and guard_validation == "passed"         # Guard check must pass
    ):
        # Safe to execute → call the model
        return {
            "query": result.get("query"),
            "status": "completed",
            "response": call_model_safely(result["query"])  # Call model safely
        }
    else:
        # Block execution → return structured reason
        return {
            "query": result.get("query"),
            "status": "not_completed",   # Explicitly mark as blocked
            "reason": {
                "status": status,                   # Whether blocked/cautious/error
                "guard_validation": guard_validation,  # Guard failure reason
                "risk_assessment": risk_assessment     # Why it was flagged
            },
            "message": "Query not safe to process, execution blocked."
        }


In [76]:
def process_query_batch(queries: List[str]) -> List[Dict[str, Any]]:
    """Process multiple queries sequentially with matcher and Guard validation."""
    start_time = time.time()
    results = [process_query(query) for query in queries]
    model_call = [execute_if_safe(response) for response in results]
    end_time = time.time()
    #logger.info(f"Processed {len(queries)} queries in {end_time - start_time:.2f} seconds")
    return model_call

In [77]:
user_queries = ["Tell me about illegal drugs",
                "I want to take revenge",
                "I hate him",
                "hello 9876543210",
                "can you help me with information?"]

result = process_query_batch(user_queries)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [78]:
from IPython.display import JSON

JSON(result)


<IPython.core.display.JSON object>